In [1]:
import pprint, json, math, os, sys
import fitz
import pandas as pd
from collections import defaultdict
import pdfminer

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
sys.path.append(os.path.abspath(dir_path))

from app.fundData import *
from app.helper import Helper


dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'


mutual_fund = Helper.get_fund_paths(fund_path)

In [2]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        print(f"___{pgn}___")
        for num,block in enumerate(blocks.get('blocks',[])):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if num in range(3,10):
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
    doc = fitz.open(path)
    final_fund_names = dict()
    
    for pgn in range(doc.page_count):
        text_all = ''
        if pgn in pages:
            # print(pgn)
            page = doc[pgn]            
            blocks = page.get_text("dict")['blocks']
            
            sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
            for count,block in enumerate(sorted_blocks):
                for line in block.get("lines", []):
                    for span in line.get("spans", []):
                        text = span['text'].strip()
                        if count in range(0,1):
                            text_all+=f" {text}"
            print(text_all)
        pattern = r"MIRAE ASSET.*?\b(?:ETF|EOF|FOF|FTF|FUND)\b"
        if matches := re.findall(pattern, text_all.strip(), re.DOTALL):
            final_fund_names[pgn] = matches[0]
        else:
            final_fund_names[pgn] = ""
    return final_fund_names

In [24]:
# Lines to be drawn (start and end points)
lines = [
    ((290, 0), (290, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0, 65, 290, 812)]

sample_path  = mutual_fund['PPFAS Mutual Fund']
pages = [i for i in range(1,70)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)


Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\DryRun.pdf


In [19]:
data = Helper.get_all_pdf_data(sample_path)

In [26]:
data[1:12]

[{'pgn': 1,
  'blocks': [{'number': 21,
    'type': 0,
    'bbox': (401.58099365234375,
     2.6312618255615234,
     566.2410278320312,
     13.907261848449707),
    'lines': [{'spans': [{'size': 11.276000022888184,
        'flags': 20,
        'font': 'VAGRoundedLTPro-Bold',
        'color': -65794,
        'ascender': 0.7699999809265137,
        'descender': -0.23000000417232513,
        'text': 'FACT SHEET - DECEMBER 2024',
        'origin': (401.58099365234375, 11.31378173828125),
        'bbox': (401.58099365234375,
         2.6312618255615234,
         566.2410278320312,
         13.907261848449707)}],
      'wmode': 0,
      'dir': (1.0, 0.0),
      'bbox': (401.58099365234375,
       2.6312618255615234,
       566.2410278320312,
       13.907261848449707)}]},
   {'number': 24,
    'type': 0,
    'bbox': (95.64070129394531,
     5.165681838989258,
     306.3784484863281,
     31.73057746887207),
    'lines': [{'spans': [{'size': 7.369825839996338,
        'flags': 4,
        'f